In [17]:
# !pip install transformers datasets
# !pip install transformers[torch]
# !pip install accelerate -U
# ! pip install -U transformers

In [18]:
# Imports
import torch
import pandas as pd
from transformers import BertTokenizerFast, BertModel
from torch.utils.data import Dataset
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from string import punctuation
import re
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
# Set up GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/IMDB_Dataset.csv' # @Team: Replace with your own google drive path to dataset
df = pd.read_csv(file_path)
df = df.sample(frac=1.0, random_state=413)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,review,sentiment
21315,"First of all, it is interesting to note that o...",positive
20835,The unthinkable has happened. Having first wit...,negative
29274,One of the best records of Israel's response t...,positive
32234,"But, lets face it... it got a few nostalgic si...",negative
26597,Ben a out-of-town cop is convinced his sister ...,negative


In [21]:
# Convert sentiment to binary labels
df.rename(columns={'sentiment': 'labels'}, inplace=True)
label_mapping = {'positive': 1, 'negative': 0}
df['labels'] = df['labels'].map(label_mapping)
df.head()

,review,labels
21315,"First of all, it is interesting to note that o...",1
20835,The unthinkable has happened. Having first wit...,0
29274,One of the best records of Israel's response t...,1
32234,"But, lets face it... it got a few nostalgic si...",0
26597,Ben a out-of-town cop is convinced his sister ...,0


In [22]:
# Function to clean data
def clean_data(text):
    english_stopwords = set(stopwords.words("english"))
    cleaned_text = []
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(f'[{re.escape(punctuation)}]', ' ', text)
    for token in text.split():
        if token.lower() not in english_stopwords and not token.isdigit():
            cleaned_text.append(token.lower())
    return ' '.join(cleaned_text)

# Apply clean_data function to 'review' column
df['review'] = df['review'].apply(lambda x: clean_data(x))

df.head()

,review,labels
21315,first interesting note one users commented fil...,1
20835,unthinkable happened first witnessed years ago...,0
29274,one best records israel response murder rabin ...,1
32234,lets face got nostalgic sighs show consistentl...,0
26597,ben town cop convinced sister brutally killed ...,0


In [23]:
# Import BERT Tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=2048)

In [46]:
from torch import nn
class MyTransformer(nn.Module):
    def __init__(self, hidden_size=64):
      super(MyTransformer, self).__init__()
      self.hidden = hidden_size
      self.bert = BertModel.from_pretrained("bert-base-uncased")
      self.lstm = nn.LSTM(768, self.hidden, num_layers=3, bidirectional=True, batch_first=True)
      self.fc = nn.Linear(self.hidden * 2, 2)


    def forward(self, input_ids, attention_mask, labels=None):
      outputs = self.bert(input_ids, attention_mask=attention_mask)
      sequence_output = outputs.last_hidden_state
      lstm_output, (hn, cn) = self.lstm(sequence_output)
      lstm_output = lstm_output[:, -1, :]
      logits = self.fc(lstm_output)

      outputs = (logits,) + outputs[2:]

      if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))
            outputs = (loss,) + outputs

      return outputs

transformer_model = MyTransformer()
transformer_model = transformer_model.to(device)

In [26]:
# Split dataset into 70% training, 10% validation, 20% testing
train_texts = list(df.review[:35000])
val_texts = list(df.review[35000:40000])
test_texts = list(df.review[40000:])

train_labels = list(df.labels[:35000])
val_labels = list(df.labels[35000:40000])
test_labels = list(df.labels[40000:])

In [27]:
# Tokenize different datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [28]:
# Custom dataset
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [29]:
# Instantiate custom datasets
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [30]:
# Compute accuracy and metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)

    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

In [47]:
# Training arguments with lr = 6e-5
training_args6 = TrainingArguments(
    output_dir='./',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=30,
    logging_steps=50,
    evaluation_strategy="steps",
    learning_rate=6e-5,
    eval_steps=50,
    fp16=True,
    load_best_model_at_end=True
)

# Define trainer with arguments from above
trainer6 = Trainer(
    model=transformer_model,
    args=training_args6,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics= compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [48]:
# Fine-tune pretrained Bert Model
trainer6.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.646000,0.554942,0.807800,0.827893,0.807403,0.804669
100,0.532800,0.458071,0.845400,0.845742,0.845450,0.845374
150,0.470900,0.415936,0.853400,0.855782,0.853531,0.853186
200,0.433900,0.425090,0.841000,0.849536,0.841250,0.840091
250,0.478300,0.403060,0.822800,0.842656,0.823185,0.820315
300,0.432000,0.368875,0.866000,0.866192,0.866037,0.865990
350,0.401500,0.342739,0.865000,0.867911,0.865143,0.864764
400,0.419900,0.340986,0.873200,0.873395,0.873237,0.873190
450,0.370900,0.359965,0.851600,0.859579,0.851362,0.850710
500,0.392800,0.436465,0.800600,0.845219,0.801175,0.794151


TrainOutput(global_step=5250, training_loss=0.2288379547482445, metrics={'train_runtime': 1975.7697, 'train_samples_per_second': 53.144, 'train_steps_per_second': 2.657, 'total_flos': 0.0, 'train_loss': 0.2288379547482445, 'epoch': 3.0})

In [49]:
# Record test results
results6 = trainer6.predict(test_dataset)
results6

PredictionOutput(predictions=array([[-1.2275391 ,  1.3066406 ],
       [-0.5029297 ,  0.4663086 ],
       [-1.4980469 ,  1.6796875 ],
       ...,
       [-0.13916016,  0.07775879],
       [ 1.1230469 , -1.0292969 ],
       [-1.4882812 ,  1.6640625 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 1]), metrics={'test_loss': 0.25526636838912964, 'test_accuracy': 0.8997, 'test_precision': 0.8998721879057916, 'test_recall': 0.8996457736707603, 'test_f1': 0.8996777274522716, 'test_runtime': 22.781, 'test_samples_per_second': 438.963, 'test_steps_per_second': 14.661})